In [1]:
import pandas as pd
import numpy as np
import random

#split dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#AUCROC,AUCPR,precision,recall,f1-score
from sklearn.metrics import roc_curve,auc,average_precision_score,precision_score,recall_score,f1_score

#gridsearch/randomsearch
from itertools import product
from tqdm import tqdm

#visualize results
import matplotlib.pyplot as plt
import time

In [2]:
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb 
import xgboost as xgb

D:\Anaconda37\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
D:\Anaconda37\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [3]:
def set_seed(seed):
  np.random.seed(seed)
  random.seed(seed)

Data

In [4]:
def data_prepare(contam_ratio,seed):
    set_seed(seed)
    data = pd.read_csv("D:/Jiang/Research_Anomaly Detection/Important_Credit Card Fraud Detection (CCFD)/creditcardfraud/creditcard.csv")
    #change 0,1 label to 1,-1
    data.loc[data['Class']==1,'Class'] = -1
    data.loc[data['Class']==0,'Class'] = 1

    X = data.drop(['Time','Class'], axis=1)
    y = data["Class"].values

    #split the data to training, validation and testing data (50%,20%,30%)
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,shuffle=False)
    X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=2/7,shuffle=False)

    #Minmax
    scaler=MinMaxScaler().fit(X_train)

    X_train=scaler.transform(X_train)
    X_val = scaler.transform(X_val)
    X_test=scaler.transform(X_test)

    index_contam = np.arange(len(y_train))[y_train == -1]
    index_contam = np.random.choice(index_contam,int(contam_ratio*len(index_contam)),replace = False)

    y_train[index_contam] = 1
    
    return X_train,y_train,X_val,y_val,X_test,y_test

In [5]:
def random_search(hyper_list_entire,search_size,seed):
  if search_size < len(hyper_list_entire):
    set_seed(seed)
    index = np.random.choice(np.arange(len(hyper_list_entire)),search_size,replace = False)

    hyper_list = []
    for i in index:
      hyper_list.append(hyper_list_entire[i])
  else:
    hyper_list = hyper_list_entire

  return hyper_list

In [6]:
def output(model_name):
  ######################################################global parameters#################################################################
  seed_pool = [1,2,3,4,5]
  anomaly_ratio_pool = [0.001,0.002,0.003]
  contam_ratio_pool = [0.98,0.8,0.5,0.0]

  #random search size
  search_size = 10

  if model_name == 'RF':
    hyper_n_estimators = [50,100,150]
    hyper_max_features = [0.6,0.8,1.0]
    hyper_list_entire = list(product(hyper_n_estimators,hyper_max_features))

  elif model_name == 'LGB':
    hyper_colsample_bytree = [0.6,0.8,1]
    hyper_learning_rate = [0.05,0.1,0.15]
    hyper_n_estimators = [50,100,150]
    hyper_reg_alpha = [0.01,0.1,1]
    hyper_reg_lambda = [0.01,0.1,1]
    hyper_list_entire = list(product(hyper_colsample_bytree,hyper_learning_rate,hyper_n_estimators,hyper_reg_alpha,hyper_reg_lambda))

  elif model_name == 'XGB':
    hyper_colsample_bytree = [0.6,0.8,1]
    hyper_learning_rate = [0.05,0.1,0.15]
    hyper_n_estimators = [50,100,150]
    hyper_reg_alpha = [0.01,0.1,1]
    hyper_reg_lambda = [0.01,0.1,1]
    hyper_list_entire = list(product(hyper_colsample_bytree,hyper_learning_rate,hyper_n_estimators,hyper_reg_alpha,hyper_reg_lambda))

  else:
    print('error model name!')


  for contam_ratio in contam_ratio_pool:
    df_result = pd.DataFrame(data = None,index = ['AUCPR'] + anomaly_ratio_pool,columns = seed_pool)
    for seed in tqdm(seed_pool):
      #############################################seleting the best hyper-parameters in validation set#############################################
      metric_value_list=list()
      hyper_list = random_search(hyper_list_entire,search_size,seed)

      #data
      X_train,y_train,X_val,y_val,X_test,y_test = data_prepare(contam_ratio,seed)
      
      for i in tqdm(range(len(hyper_list))):
        try:
          print(f'Finding Optimal Hyper-parameters......Current Candidates: {hyper_list[i]}')
          if model_name == 'RF':
            n_estimators,max_features = hyper_list[i]
            model = RandomForestClassifier(n_estimators = n_estimators,max_features = max_features,random_state = seed)
          elif model_name == 'LGB':
            colsample_bytree,learning_rate,n_estimators,reg_alpha,reg_lambda = hyper_list[i]
            model = lgb.LGBMClassifier(colsample_bytree = colsample_bytree,learning_rate = learning_rate, n_estimators = n_estimators,
                      reg_alpha = reg_alpha, reg_lambda = reg_lambda, random_state = seed)
          elif model_name == 'XGB':
            colsample_bytree,learning_rate,n_estimators,reg_alpha,reg_lambda = hyper_list[i]
            model = xgb.XGBClassifier(colsample_bytree = colsample_bytree,learning_rate = learning_rate, n_estimators = n_estimators,
                      reg_alpha = reg_alpha, reg_lambda = reg_lambda, random_state = seed)
            
          model.fit(X_train,y_train)

          #evaluation
          score = model.predict_proba(X_val)[:,0]
          metric_value = average_precision_score(y_true = y_val,y_score = score,pos_label = -1)
          metric_value_list.append(metric_value)

          print(f'The metric value corresponded to the hyper-parameters is :{metric_value:{.4}}')
          print('******************************')
          print('\n')
        except:
          pass
        continue

      best_hyper_params = hyper_list[metric_value_list.index(max(metric_value_list))]
      print(f'The best hyper-parameters are: {best_hyper_params}')
      print('\n')
      ###################################################################testing#########################################################################
      print('Testing Phrase......')
      if model_name == 'RF':
        n_estimators,max_features = best_hyper_params
        model = RandomForestClassifier(n_estimators = n_estimators,max_features = max_features,random_state = seed)
      elif model_name == 'LGB':
        colsample_bytree,learning_rate,n_estimators,reg_alpha,reg_lambda = best_hyper_params
        model = lgb.LGBMClassifier(colsample_bytree = colsample_bytree,learning_rate = learning_rate, n_estimators = n_estimators,
                    reg_alpha = reg_alpha, reg_lambda = reg_lambda, random_state = seed)
      elif model_name == 'XGB':
        colsample_bytree,learning_rate,n_estimators,reg_alpha,reg_lambda = best_hyper_params
        model = xgb.XGBClassifier(colsample_bytree = colsample_bytree,learning_rate = learning_rate, n_estimators = n_estimators,
                    reg_alpha = reg_alpha, reg_lambda = reg_lambda, random_state = seed)
      model.fit(X_train,y_train)

      #evaluation
      score = model.predict_proba(X_test)[:,0]
      
      #store the result
      #AUCPR
      df_result.loc['AUCPR',seed] = average_precision_score(y_true = y_test,y_score = score,pos_label = -1)
      #F1
      for anomaly_ratio in anomaly_ratio_pool:
          threshold = score[np.argsort(-score)][int(anomaly_ratio*len(score))]
          
          y_pred = np.ones(len(score))
          y_pred[score >= threshold] = -1
          
          print('\n')
          print(f'Precision: {round(precision_score(y_pred = y_pred, y_true = y_test, pos_label= -1)*100,2)}')
          print(f'Recall: {round(recall_score(y_pred = y_pred, y_true = y_test, pos_label= -1)*100,2)}')
          print(f'F1-score: {round(f1_score(y_pred = y_pred, y_true = y_test, pos_label= -1)*100,2)}')
          print('\n')

          df_result.loc[anomaly_ratio,seed] = f1_score(y_pred = y_pred,y_true = y_test,pos_label = -1) 

    #mean & sd
    df_result['mean'] = np.mean(df_result.loc[:,seed_pool],axis = 1)
    df_result['std'] = np.std(df_result.loc[:,seed_pool],axis = 1)
    df_result = round(df_result.astype('float64')*100,2)

    file_path = 'D:/Jiang/Research_Anomaly Detection/Important_Credit Card Fraud Detection (CCFD)/Hyprid Semi-supervised/result/CCFD_' +\
                model_name + '_' + str(contam_ratio) + '.csv'
    df_result.to_csv(file_path,index = False)

In [7]:
output('RF')
output('LGB')
output('XGB')

  0%|                                                                                            | 0/9 [00:00<?, ?it/s]

Finding Optimal Hyper-parameters......Current Candidates: (50, 0.6)
The metric value corresponded to the hyper-parameters is :0.3649
******************************





 11%|█████████▎                                                                          | 1/9 [00:31<04:11, 31.41s/it]

Finding Optimal Hyper-parameters......Current Candidates: (50, 0.8)
The metric value corresponded to the hyper-parameters is :0.4638
******************************





 22%|██████████████████▋                                                                 | 2/9 [01:13<04:01, 34.48s/it]

...


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:16<00:09,  9.51s/it]

Finding Optimal Hyper-parameters......Current Candidates: (0.8, 0.05, 50, 1, 0.1)
The metric value corresponded to the hyper-parameters is :0.7698
******************************





100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:20<00:00,  7.74s/it]

The best hyper-parameters are: (0.6, 0.1, 150, 1, 0.01)


Testing Phrase......


Precision: 81.4
Recall: 64.81
F1-score: 72.16




Precision: 50.29
Recall: 79.63
F1-score: 61.65




Precision: 33.46
Recall: 79.63
F1-score: 47.12




100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [07:29<00:00, 89.17s/it]
